# 환경설정

In [5]:
## setup(1min)
# for ColossalAI
!pip install colossalai==0.2.7

# setup data
!git clone https://github.com/airobotlab/KoChatGPT
!mv KoChatGPT/data_kochatgpt .
!mv KoChatGPT/img .

%cd KoChatGPT/colossalai_ChatGPT_230319/
!pip install .
%cd ../../

# setup library
!pip install openai
!pip install langchain==0.0.113
!pip install pandas>=1.4.1

!pip uninstall -y transformers
!pip install transformers==4.28.0

!pip install datasets
!pip install xlrd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.7/686.7 kB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

Cloning into 'KoChatGPT'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 249 (delta 67), reused 96 (delta 51), pack-reused 129
Receiving objects: 100% (249/249), 37.35 MiB | 13.94 MiB/s, done.
Resolving deltas: 100% (97/97), done.
/content/KoChatGPT/colossalai_ChatGPT_230319
Processing /content/KoChatGPT/colossalai_ChatGPT_230319
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 90.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
# import
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead
from copy import deepcopy
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer
from transformers import GPT2LMHeadModel
from transformers import PreTrainedTokenizerFast
import pandas as pd
import argparse
import copy
import logging
import json
from dataclasses import dataclass, field

In [7]:
# define argment
parser = argparse.ArgumentParser()
parser.add_argument('--data_path_1_SFT', type=str, default='/content/data_kochatgpt/kochatgpt_1_SFT.jsonl')
# parser.add_argument('--data_path_1_SFT', type=str, default='./SFT_dataset/')
parser.add_argument('--model_name', type=str, default='gpt2', choices=['gpt2', 'bloom', 'opt'])
parser.add_argument('--max_epochs', type=int, default=2)
parser.add_argument('--train_batch_size', type=int, default=8)
parser.add_argument('--output_dir', type=str, default='./output_1_SFT')

args = parser.parse_args(args=[])

# for test
args.model_name = 'skt/kogpt2-base-v2'

In [8]:
## test & load skt gpt2 kroean
import torch
from transformers import GPT2LMHeadModel

from transformers import PreTrainedTokenizerFast

In [9]:
# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n"
        "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n"
        "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{prompt}\n\n### Input(입력):\n{input}\n\n### Response(응답):"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n"
        "아래는 작업을 설명하는 명령어입니다.\n\n"
        "Write a response that appropriately completes the request.\n명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    ),
}

In [10]:
## 모델 준비
model = AutoModelForCausalLM.from_pretrained(args.model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    args.model_name,
    padding_side="right",
    model_max_length=1024,
)
tokenizer.add_special_tokens(
    {
        "eos_token": DEFAULT_EOS_TOKEN,
        "bos_token": DEFAULT_BOS_TOKEN,
        "unk_token": DEFAULT_UNK_TOKEN,
    }
)
tokenizer.pad_token = tokenizer.eos_token
print(tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True)


# 데이터 전처리

In [12]:
## prepare data
from typing import Optional, Dict, Sequence
import random
import re

!pip install openpyxl
test_data = []

class SFT_dataset(Dataset):
    '''SFT dataset by wygo'''
    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, mode: str, verbose=True, test=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        ## format
        pattern_instruction = 'prompt'  # instruction
        pattern_input = 'input'  # 내 데이터엔 input이 없다
        pattern_output = 'completion'  # output

        ## 데이터셋 만들기, source와 target
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]  # 템플릿 가져오기
        list_data_dict = []

        # summary mode
        if mode == 'summary':

            # 프롬포트 설정
            prompt =  "아래의 텍스트는 리뷰 텍스트입니다.\n"\
                      "이 텍스트에서 리뷰 작성자가 느낀 감성의 원인이 잘 드러나도록 요약문을 제시해주세요.\n"\
                      "요약문은 리뷰 작성자가 리뷰 대상이 왜 좋았는지, 싫었는지를 잘 설명해야 합니다.\n"\
                      "요약문의 시작은 [ 로, 끝은 ]로 제시하세요."

            # 엑셀 데이터 셋 사용
            df = pd.read_excel(data_path_1_SFT)
            df.dropna(inplace=True)
            df.reset_index(drop=True, inplace=True)
            list_data_dict = []

            # 데이터 전처리
            for i in df.index:
                mission = df.iloc[i]['input']
                summary = '[' + df.iloc[i]['completion'] + ']'

                tmp = {'prompt': prompt, 'input': mission, 'completion': summary}
                list_data_dict.append(tmp)

        # key phrase mode
        if mode == 'keyphrase':

            # 프롬포트 설정
            prompt =  "아래의 텍스트는 리뷰 텍스트입니다.\n"\
                      "이 텍스트에서 리뷰 작성자가 느낀 감성의 원인이 되는 핵심적인 구문을 제시해주세요.\n"\
                      "각 핵심 구문은 리뷰 작성자가 리뷰 대상이 왜 좋았는지, 싫었는지를 잘 설명해야 합니다.\n"\
                      "핵심 구문이 여러 개일 경우, • 로 구분해서 제시하세요."

            # 엑셀 데이터 셋 사용
            df = pd.read_excel(data_path_1_SFT)
            df.dropna(inplace=True)
            df.reset_index(drop=True, inplace=True)
            list_data_dict = []

            # 데이터 전처리
            for i in df.index:
                mission = df.iloc[i]['input']
                keywords = df.iloc[i]['completion'].split('• ')
                keywords = [keyword.strip().replace('\n', ' ') for keyword in keywords if keyword.strip()]
                keywords = ["• " + keyword.strip() for keyword in keywords]
                keywords = [re.sub(r'\s{2,}', ' ', keyword) for keyword in keywords]
                keywords_str = str(keywords)[1:-1]

                tmp = {'prompt': prompt, 'input': mission, 'completion': keywords_str}
                list_data_dict.append(tmp)

        # sentiment mode
        if mode == 'sentiment':

            prompt = "아래의 텍스트를 분석하여 긍정/부정을 판단해줘\n"
            df = pd.read_excel("./good_or_bad.xlsx")
            df.dropna(inplace=True)
            df.reset_index(drop = True, inplace=True)
            for i in df.index:
              mission = df.iloc[i]['input']
              sentiment = df.iloc[i]['PosOrNeg']
              tmp = {'prompt': prompt, 'input': mission, 'completion': sentiment}
              if i % 6 != 0 and test == False:
                list_data_dict.append(tmp)
              elif i % 6 == 0 and test == True:
                test_data.append(tmp)

            df = pd.read_excel("./naver_webboard_total.xlsx")
            df.dropna(inplace=True)
            df.reset_index(drop = True, inplace=True)
            for i in df.index:
              mission = df.iloc[i]['input']
              sentiment = df.iloc[i]['PosOrNeg']
              tmp = {'prompt': prompt, 'input': mission, 'completion': sentiment}
              if i % 6 != 0 and test == False:
                list_data_dict.append(tmp)
              elif i % 6 == 0 and test == True:
                test_data.append(tmp)


        #print(list_data_dict[0])
        random.shuffle(list_data_dict)
        #print(list_data_dict[0])
        # 입력
        sources = []
        for example in list_data_dict:
            if example.get(pattern_input, "") != "":
                tmp = prompt_input.format_map(example)
            else:
                tmp = prompt_no_input.format_map(example)
            sources.append(tmp)

        # 출력
        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")

        if verbose:
            idx = 80
            print((sources[idx]))
            print((targets[idx]))
            print("Tokenizing inputs... This may take some time...")

        ############################################################
        # data_dict = preprocess(sources, targets, tokenizer)  # https://github.com/Beomi/KoAlpaca/blob/04704348d58b8b1c2e2638d6437a04b4e8ba1823/train.py#L124
        examples = [s + t for s, t in zip(sources, targets)]

        # source data tokenized
        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source만
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        ## 입력은 source, 출력은 source+target 이지만 학습은 target 부분만
        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = IGNORE_INDEX  # source 부분은 -100으로 채운다

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))

    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        """Tokenize a list of strings."""
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest", #패딩을 가장 긴 시퀀스에 맞춤
                max_length=tokenizer.model_max_length,
                truncation=True, #토큰화된 시퀀스가 최대 길이를 초과할 경우 자르기
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]

        # 최대 길이 이상으로 잘렸는지 확인
        #for tokenized in tokenized_list:
        #  print('%d %d'% (len(tokenized['input_ids'][0]), tokenizer.model_max_length))

        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self): #샘플 개수
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]: #샘플 가져오기
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        #print(self.tokenizer.pad_token_id)
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )



#train_dataset = SFT_dataset(data_path_1_SFT='/content/keyphrase_traindata.xls', tokenizer=tokenizer, mode='keyphrase')
train_dataset = SFT_dataset(data_path_1_SFT='/content/summary_traindata.xls', tokenizer=tokenizer, mode='summary')


data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)


Below is an instruction that describes a task, paired with an input that provides further context.
아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.

Write a response that appropriately completes the request.
요청을 적절히 완료하는 응답을 작성하세요.

### Instruction(명령어):
아래의 텍스트는 리뷰 텍스트입니다.
이 텍스트에서 리뷰 작성자가 느낀 감성의 원인이 잘 드러나도록 요약문을 제시해주세요.
요약문은 리뷰 작성자가 리뷰 대상이 왜 좋았는지, 싫었는지를 잘 설명해야 합니다.
요약문의 시작은 [ 로, 끝은 ]로 제시하세요.

### Input(입력):
처음엔 혼자 공부하는 방법도 잘 모르겠었구, 다른 친구들이 PMP로 인강 듣는걸 보고, 저도 인강을 들으려 했어요. 왠지 혼자 공부하는 것보단 도움이 조금이라도 더 될것 같은 기분에. 좋은 인강 사이트가 무엇이 있을까 하다가 가장 보편적일 것 같았던 EBS로 결정하고 과학부터 들었는데요.
굉장히 낯설었어요. 예전에 몇번 경험한 적은(본 적은) 있었지만, 아무래도 교실 안에서 선생님과의 호흡이 느껴지는 느낌이 나지 않아서 그랬던 것 같아요. 그래도 강의 해주시는 선생님들이 열심히 해주시는 것 같아 이해도 잘되고 좋았구요. 
아, 그리고 저희 집 컴퓨터에 꼭 켜 놓는 프로그램이 하나 있는데 그게 수시로 봐주고 점검을 해주어야 되거든요. 근데, 인강 듣다가 그거 점검하고 오면 화면이 다른데로 사라져 버려요~ 그런 것 되게 불편했어요. 그래서 그 프로 안 켜놓을 때만 인강 들으려고 하니, 계속 새벽에만 들었어요 T.T~

(그 때 들었던 과학 인강이 없어져버린 듯 해서 다른 과학 인강 이미지 첨부 할게요~)  

### Response(응답):
[PMP로 인강을 듣기로 결정하여 EBS를 선택하여 과학 인강을 들었으나, 선생님과의 호흡이 느껴지지

# 학습 시작

In [13]:
def safe_save_model_for_hf_trainer(trainer: transformers.Trainer, output_dir: str):
    """Collects the state dict and dump to disk."""
    state_dict = trainer.model.state_dict()
    if trainer.args.should_save:
        cpu_state_dict = {key: value.cpu() for key, value in list(state_dict.items())}
        del state_dict
        trainer._save(output_dir, state_dict=cpu_state_dict)  # noqa

In [ ]:
# training_args 수정 가능: https://github.com/Beomi/KoAlpaca/blob/main/train.sh 참고
training_args = TrainingArguments(
    output_dir="./test", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=16, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 3, # Number of update steps between two evaluations.
    save_steps=500, # after # steps model is saved
    warmup_steps=10,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    optim="adamw_torch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=None,
)

trainer.train()
trainer.save_state()
safe_save_model_for_hf_trainer(trainer=trainer, output_dir=args.output_dir)

# 모델 테스트

In [ ]:
import re

def format_output_strings_keyphrase(list_result2):
    def remove_pattern(text):
        pattern = re.compile("[\u4e00-\u9fff]+")
        result = re.sub(pattern, "", str(text))
        return result

    # 'Response(응답)' 다음의 문자열 찾기
    response_index = list_result2.find('Response(응답):')

    modified_string = remove_pattern(list_result2[response_index + 13:])

    completions = []
    pattern = r"'([^']*)•([^']*)'"
    matches = re.findall(pattern, modified_string)
    for match in matches:
        completion = match[0] + '•' + match[1]
        completions.append(completion)

    return list_result2[:response_index + 13], ', '.join(completions)

In [ ]:
import re

def format_output_strings_summary(list_result2):
    def remove_pattern(text):
        pattern = re.compile("[\u4e00-\u9fff]+")
        result = re.sub(pattern, "", str(text))
        return result

    # 'Response(응답)' 다음의 문자열 찾기
    response_index = list_result2.find('Response(응답):')
    if response_index == -1:
        # 'Response(응답)'가 없으면 원본 문자열 그대로 반환
        return list_result2

    modified_string = remove_pattern(list_result2[response_index + 13:])

    match = re.search(r'\[(.*?)\]', modified_string)
    if match:
        extracted_part = match.group(1)
    else:
        extracted_part = ""

    return list_result2[:response_index + 13], list_result2[response_index + 13:], extracted_part

In [ ]:
!pip install xlwt

In [ ]:
## 추론 테스트
mode = 'text-generation'
# mode = 'sentiment-analysis'

generator = pipeline(mode, model=args.output_dir, tokenizer=tokenizer)

generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375,
    max_new_tokens=128,
    do_sample=True,
    top_k=50,
)

df = pd.read_excel('summary_testdata.xls')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

list_input2 = df['input'].tolist()

#prompt2 =  "아래의 텍스트는 제품 리뷰 텍스트입니다.\n"\
#           "이 텍스트에서 리뷰 작성자가 느낀 감성의 원인이 되는 핵심적인 구문을 제시해주세요.\n"\
#           "각 핵심 구문은 리뷰 작성자가 제품이 왜 좋았는지, 싫었는지를 잘 설명해야 합니다.\n"\
#           "핵심 구문이 여러 개일 경우, • 로 구분해서 제시하세요.\n"

prompt2 = "아래의 텍스트는 리뷰 텍스트입니다.\n"\
          "이 텍스트에서 리뷰 작성자가 느낀 감성의 원인이 잘 드러나도록 요약문을 제시해주세요.\n"\
          "요약문은 리뷰 작성자가 리뷰 대상이 왜 좋았는지, 싫었는지를 잘 설명해야 합니다.\n"\
          "요약문의 시작은 [ 로, 끝은 ]로 제시하세요."

list_input2 = [PROMPT_DICT['prompt_input'].format_map({'prompt': prompt2,'input' : tmp}) for tmp in list_input2]
list_result2 = generator(list_input2, **generation_args)

data = []
for prompt, result in zip(list_input2, list_result2):
    input, original_result, formatted_result = format_output_strings_summary(result[0]['generated_text'])
    data.append((input, original_result, formatted_result))
    #print(formatted_text_result)
    #formatted_text = format_output_strings_summary(result[0]['generated_text'])

df = pd.DataFrame(data, columns=['input', 'original_result', 'formatted_result'])

# 엑셀 파일로 저장
df.to_excel('output_data.xls', index=False)

##### Step 2) Train the reward model
-[ref](https://github.com/hpcaitech/ColossalAI/blob/main/applications/ChatGPT/examples/train_reward_model.py)

We use rm-static as dataset to train our reward model.
It is a dataset of chosen & rejected response of the same prompt.
You can download the dataset from huggingface automatically.
Use these code to train your reward model.

##### Naive reward model training
python train_reward_model.py --pretrain <your model path>

##### if to use LoRA
python train_reward_model.py --pretrain <your model path> --lora_rank 16

In [ ]:
## setup(1min)
# for ColossalAI
!pip install colossalai==0.2.7

# setup data
!git clone https://github.com/airobotlab/KoChatGPT
!mv KoChatGPT/data_kochatgpt .
!mv KoChatGPT/img .

%cd KoChatGPT/colossalai_ChatGPT_230319/
!pip install .
%cd ../../

In [ ]:
!pip install --upgrade torch==1.13.1

In [ ]:
# import
import argparse

import loralib as lora
import torch
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.models.bloom import BLOOMRM
from chatgpt.models.gpt import GPTRM
from chatgpt.models.opt import OPTRM
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import ColossalAIStrategy, DDPStrategy, NaiveStrategy
from datasets import load_dataset
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer

from colossalai.nn.optimizer import HybridAdam

import os
import json

# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n"
        "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n"
        "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{prompt}\n\n### Input(입력):\n{input}\n\n### Response(응답):"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n"
        "아래는 작업을 설명하는 명령어입니다.\n\n"
        "Write a response that appropriately completes the request.\n명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    ),
}

In [ ]:
# define argment
parser = argparse.ArgumentParser()
parser.add_argument('--output_dir', type=str, default='./output_2_RM')
parser.add_argument('--data_path_2_RM', type=str, default='./data_kochatgpt/kochatgpt_2_RM.jsonl', help='https://huggingface.co/datasets/fka/awesome-chatgpt-prompts/blob/main/prompts.csv')
parser.add_argument('--strategy',
                    choices=['naive', 'ddp', 'colossalai_gemini', 'colossalai_zero2'],
                    default='naive')
parser.add_argument('--model', type=str, default='gpt2', choices=['gpt2', 'bloom', 'opt'])
parser.add_argument('--pretrain', type=str, default=None)
parser.add_argument('--dataset', type=str, default='Dahoas/rm-static')
parser.add_argument('--save_path', type=str, default='rm_ckpt.pth')
parser.add_argument('--max_epochs', type=int, default=10)
parser.add_argument('--batch_size', type=int, default=4)
parser.add_argument('--lora_rank', type=int, default=0, help="low-rank adaptation matrices rank")
parser.add_argument('--max_len', type=int, default=512)  # wygo 추가

args = parser.parse_args(args=[])

# for test
args.max_epochs = 1
args.pretrain = 'skt/kogpt2-base-v2'  # pretrained 모델 가져오기
args.verbose = True

print(args)
if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

In [ ]:
# configure strategy
if args.strategy == 'naive':
    strategy = NaiveStrategy()
elif args.strategy == 'ddp':
    strategy = DDPStrategy()
elif args.strategy == 'colossalai_gemini':
    strategy = ColossalAIStrategy(stage=3, placement_policy='cuda')
elif args.strategy == 'colossalai_zero2':
    strategy = ColossalAIStrategy(stage=2, placement_policy='cuda')
else:
    raise ValueError(f'Unsupported strategy "{args.strategy}"')

In [ ]:
# customizing, https://github.com/hpcaitech/ColossalAI/blob/2e16f842a9e5b1fb54e7e41070e9d2bb5cd64d7c/applications/ChatGPT/chatgpt/nn/gpt_rm.py#L29
from typing import Optional

import torch.nn as nn
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model

# from ..base import RewardModel
from chatgpt.models.base import RewardModel


class GPTRM_custom(RewardModel):
    """
    GPT Reward model.
    Args:
        pretrained (str): Pretrained model name or path.
        config (GPT2Config): Model config.
        checkpoint (bool): Enable gradient checkpointing.
        lora_rank (int): Rank of the low-rank approximation.
        lora_train_bias (str): LoRA bias training mode.
    """

    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained)
            model.resize_token_embeddings(len(tokenizer))  # wygo 추가!!!
        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())
        if checkpoint:
            model.gradient_checkpointing_enable()

        # model = model.resize_token_embeddings(len(tokenizer))

        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head, lora_rank, lora_train_bias)

In [ ]:
# configure model, tokenizer
with strategy.model_init_context():
    # load pretrained gpt2
    if args.model == 'gpt2':
#         tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        # tokenizer = AutoTokenizer.from_pretrained(args.pretrain)
        tokenizer = AutoTokenizer.from_pretrained(args.pretrain, padding_side="right", model_max_length=512)
        tokenizer.add_special_tokens(
            {
                "eos_token": DEFAULT_EOS_TOKEN,
                "bos_token": DEFAULT_BOS_TOKEN,
                "unk_token": DEFAULT_UNK_TOKEN,
            }
        )
        tokenizer.pad_token = tokenizer.eos_token
        model = GPTRM_custom(pretrained=args.pretrain, lora_rank=args.lora_rank, tokenizer=tokenizer).cuda()

    elif args.model == 'bloom':
        model = BLOOMRM(pretrained=args.pretrain, lora_rank=args.lora_rank).cuda()
        tokenizer = BloomTokenizerFast.from_pretrained(args.pretrain)

    elif args.model == 'opt':
        model = OPTRM(pretrained=args.pretrain, lora_rank=args.lora_rank).cuda()
        tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

    else:
        raise ValueError(f'Unsupported model "{args.model}"')


    # model.resize_token_embeddings(len(tokenizer))

In [ ]:
# make ranking data to chosen, rejetced data
dict2 = {}
with open('./save_RM_total.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    if args.verbose:
        print('## data check ##')
        print((list_data_dict[0]))

total_data_ranking2chosen = []
for tmp in list_data_dict:
    tmp['prompt'] = "아래의 텍스트는 리뷰 텍스트입니다.\n"\
                    "이 텍스트에서 리뷰 작성자가 느낀 감성의 원인이 되는 핵심적인 구문을 제시해주세요.\n"\
                    "각 핵심 구문은 리뷰 작성자가 리뷰 대상이 왜 좋았는지, 싫었는지를 잘 설명해야 합니다.\n"\
                    "핵심 구문이 여러 개일 경우, • 로 구분해서 제시하세요."

    one_data_ranking2chosen = []

    # data 1) 0 VS 1
    data = {}
    data['prompt'] = tmp['prompt']
    data['input'] = tmp['input']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)


    # data 2) 0 VS 2
    data = {}
    data['prompt'] = tmp['prompt']
    data['input'] = tmp['input']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    # data 1) 1 VS 2
    data = {}
    data['prompt'] = tmp['prompt']
    data['input'] = tmp['input']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)

    total_data_ranking2chosen.extend(one_data_ranking2chosen)

print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])

In [ ]:
# prepare for data and dataset
import random
random.seed(230319)
# list_tmp = list(range(10))
print(total_data_ranking2chosen[45])
random.shuffle(total_data_ranking2chosen)
print(total_data_ranking2chosen[45])
# print(total_data_ranking2chosen[45])

# train_data = total_data_ranking2chosen[:-1000]  # 29000 학습
# eval_data = total_data_ranking2chosen[-1000:]  # 1000개만 평가

train_data = total_data_ranking2chosen[:-30]  #  학습
eval_data = total_data_ranking2chosen[-30:]  # 30개만 평가

train_dataset = RewardDataset(train_data, tokenizer, args.max_len)
eval_dataset = RewardDataset(eval_data, tokenizer, args.max_len)

# check
idx = 10
print('#'*70)
print('## prompt ##')
print(train_data[idx]['prompt'])
print('#'*70)
print('## input ##')
print(train_data[idx]['input'])
print('#'*70)
print('## chosen ##')
print(train_data[idx]['chosen'])
print('#'*70)
print('## rejected ##')
print(train_data[idx]['rejected'])

In [ ]:
# configure optimizer
if args.strategy.startswith('colossalai'):
    optim = HybridAdam(model.parameters(), lr=5e-5)
else:
    optim = Adam(model.parameters(), lr=5e-5)

In [ ]:
# batch_size here is expected to be C(k,2), k means # response of each prompt
# be limited with the format of dataset 'Dahoas/rm-static', we'd better use batch_size as 1
trainer = RewardModelTrainer(model=model,
                             strategy=strategy,
                             optim=optim,
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=args.batch_size,
                             max_epochs=args.max_epochs)

###### 참고
- [train reward model](https://github.com/hpcaitech/ColossalAI/blob/2e16f842a9e5b1fb54e7e41070e9d2bb5cd64d7c/applications/ChatGPT/chatgpt/trainer/rm.py#L68)

In [ ]:
# train!!
trainer.fit(use_lora=args.lora_rank)

## save
# save model checkpoint after fitting on only rank0
strategy.save_model(model, os.path.join(args.output_dir, 'RM.pt'), only_rank0=True)
# save optimizer checkpoint on all ranks
strategy.save_optimizer(optim,
                        os.path.join(args.output_dir, 'RM_optim_checkpoint_%d.pt' % (torch.cuda.current_device())),
                        only_rank0=False)

#### 사람의 선호도를 모방한 좋은 글 채점기 모델을 학습했습니다.  
#### ``output_2_RM`` 폴더에 학습된 모델이 저장되어 있습니다.  
#### 이제 step3) PPO 모델을 학습해볼까요?

### Step 3) 사람의 피드백을 반영하여 학습

- Further fine-tune the LLM from step 1 with the reward model and this dataset using RL (e.g. PPO)
- 배경
    - **사람의 순위를 모사한 보상모델(RM)** 의 점수가 높아지도록 학습 (31,000개)
    - 초기 모델에 비해 너무 많이 바뀌지 않도록  
    
    
    
<img src="./img/3_PPO_1.png" width="650">
    

##### RLHF
- [train_dummy](https://github.com/hpcaitech/ColossalAI/blob/main/applications/ChatGPT/examples/train_dummy.py)
- [train_prompts](https://github.com/hpcaitech/ColossalAI/blob/main/applications/ChatGPT/examples/train_prompts.py)
- [좋은설명](https://gist.github.com/JoaoLages/c6f2dfd13d2484aa8bb0b2d567fbf093)

- [dataset1](https://huggingface.co/datasets/Dahoas/rm-static)
- [dataset2](https://huggingface.co/datasets/Anthropic/hh-rlhf)

<img src="https://raw.githubusercontent.com/hpcaitech/public_assets/main/applications/chatgpt/chatgpt.png" width="500">

<img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/rlhf/rlhf.png" width="500">


- Fine-tuning 태스크를 강화학습 문제로 다음과 같이 정형화
    - Policy : 언어모델 - 프롬프트를 입력으로 받아 텍스트의 시퀀스(혹은 그 확률)를 리턴
    - Action space : 언어모델의 모든 단어 (일반적으로 5만개 분량)
    - Observation space : 가능한 인풋 토큰 시퀀스 (단어개수^시퀀스길이 이므로 엄청 큼!)
    - Reward function : 보상모델과 policy shift에 대한 제약조건의 조합으로 정의됨

<img src="img/3_PPO_2.png" width="500">

- Frozen Model과 Non-frozen(trainable) Model의 텍스트 출력 확률간 KL divergence를 계산
- trainable Model의 weight가 완전히 바뀌는 것을 방지하고 Reward Model에 말도 되지 않는 텍스트로 출력을 시작하는 것을 방지


<img src="img/3_PPO_3.png" width="500">


- PPO process
[1] 초기화를 위해 intial probs(initial output text probabilities)를 new probs(new output text probabilities)와 동일하게 만듬

- while:
    - [2] New probs와 initial probs간 ratio을 계산함
    - [3] 아래 공식에 따라 loss를 계산함.
        - loss = -min(ratio * R, clip(ratio, 0.8, 1.2) * R)
            - R = reward + KL (or 0.8*reward + 0.2*KL와 같은 weighted average)
            - clip(ratio, 0.8, 1.2) → 0.8 ≤ ratio ≤ 1.2
    - [4] Loss를 backpropagating하여 SFT Model의 weight를 업데이트함

    - [5] 새롭게 업데이트된 SFT 모델로 new probs를 계산함

    - [6] 2번부터 6번을 N 번 반복함

kl = 두 확률 분포 사이의 차이를 나타내는 지표

ratio = 현재가 이전에 비해 얼마나 개선되었는 지의 비율

r = 보상값

clip = 안정화


- [loss1](https://github.com/hpcaitech/ColossalAI/blob/1216d1e7bdf223d831895e34c01fb40df36ea9c7/applications/ChatGPT/chatgpt/experience_maker/naive.py#L7)
- [loss2](https://github.com/hpcaitech/ColossalAI/blob/1216d1e7bdf223d831895e34c01fb40df36ea9c7/applications/ChatGPT/chatgpt/models/utils.py#L31)



In [ ]:
## setup(1min)
# for ColossalAI
!pip install colossalai==0.2.7

# setup data
!git clone https://github.com/airobotlab/KoChatGPT
!mv KoChatGPT/data_kochatgpt .
!mv KoChatGPT/img .

%cd KoChatGPT/colossalai_ChatGPT_230319/
!pip install .
%cd ../../

In [ ]:
# import
import argparse
from copy import deepcopy

import pandas as pd
import torch
from chatgpt.models.base import RewardModel
from chatgpt.models.bloom import BLOOMActor, BLOOMCritic
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.models.opt import OPTActor, OPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import ColossalAIStrategy, DDPStrategy, NaiveStrategy
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer

from colossalai.nn.optimizer import HybridAdam

## wy 추가
import json
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

## clossalAI error 해결
os.environ['RANK'] = '0'
os.environ['LOCAL_RANK'] = '0'
os.environ['WORLD_SIZE'] = '2'
os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '42043'

# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n"
        "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n"
        "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{prompt}\n\n### Input(입력):\n{input}\n\n### Response(응답):"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n"
        "아래는 작업을 설명하는 명령어입니다.\n\n"
        "Write a response that appropriately completes the request.\n명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    ),
}

In [ ]:
# define argument
parser = argparse.ArgumentParser()
parser.add_argument('--data_path_3_PPO', type=str, default='./data_kochatgpt/ppo_dataset.jsonl')
parser.add_argument('--output_dir', type=str, default='./output_3_PPO')
parser.add_argument('--strategy',
                    choices=['naive', 'ddp', 'colossalai_gemini', 'colossalai_zero2'],
                    default='naive')
parser.add_argument('--model', type=str, default='gpt2', choices=['gpt2', 'bloom', 'opt'])
parser.add_argument('--pretrain', type=str, default=None)
parser.add_argument('--num_episodes', type=int, default=10)
parser.add_argument('--max_timesteps', type=int, default=3)
parser.add_argument('--update_timesteps', type=int, default=3)
parser.add_argument('--max_epochs', type=int, default=5)
parser.add_argument('--train_batch_size', type=int, default=8)
parser.add_argument('--lora_rank', type=int, default=0, help="low-rank adaptation matrices rank")
parser.add_argument('--max_length', type=int, default=384)
args = parser.parse_args(args=[])

# for test
args.output_dir = './output_3_PPO'
args.pretrain = 'skt/kogpt2-base-v2'  # pretrained 모델 가져오기
args.pretrain_actor = './output_1_SFT'  # SFT 모델 가져오기
args.pretrain_critic = './output_2_RM'  # RM 모델 가져오기
# args.pretrain_actor = args.pretrain
# args.pretrain_critic = args.pretrain

args.num_episodes = 1
args.max_epochs   = 1

print(args)
if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

In [ ]:
# configure strategy
if args.strategy == 'naive':
    strategy = NaiveStrategy()
elif args.strategy == 'ddp':
    strategy = DDPStrategy()
elif args.strategy == 'colossalai_gemini':
    strategy = ColossalAIStrategy(stage=3, placement_policy='cuda')
elif args.strategy == 'colossalai_zero2':
    strategy = ColossalAIStrategy(stage=2, placement_policy='cuda')
else:
    raise ValueError(f'Unsupported strategy "{args.strategy}"')

In [ ]:
# configure model, tokenizer
with strategy.model_init_context():
    if args.model == 'gpt2':
        actor = GPTActor(pretrained=args.pretrain_actor, lora_rank=args.lora_rank).to(torch.cuda.current_device())
        critic = GPTCritic(pretrained=args.pretrain_critic, lora_rank=args.lora_rank).to(torch.cuda.current_device())
        # tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        # tokenizer.pad_token = tokenizer.eos_token
        tokenizer = AutoTokenizer.from_pretrained(args.pretrain, padding_side="right", model_max_length=512)
        tokenizer.add_special_tokens(
            {
                "eos_token": DEFAULT_EOS_TOKEN,
                "bos_token": DEFAULT_BOS_TOKEN,
                "unk_token": DEFAULT_UNK_TOKEN,
            }
        )
        tokenizer.pad_token = tokenizer.eos_token



    elif args.model == 'bloom':
        actor = BLOOMActor(pretrained=args.pretrain, lora_rank=args.lora_rank).to(torch.cuda.current_device())
        critic = BLOOMCritic(pretrained=args.pretrain, lora_rank=args.lora_rank).to(torch.cuda.current_device())
        tokenizer = BloomTokenizerFast.from_pretrained(args.pretrain)
        tokenizer.pad_token = tokenizer.eos_token
    elif args.model == 'opt':
        actor = OPTActor(pretrained=args.pretrain, lora_rank=args.lora_rank).to(torch.cuda.current_device())
        critic = OPTCritic(pretrained=args.pretrain, lora_rank=args.lora_rank).to(torch.cuda.current_device())
        tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")
    else:
        raise ValueError(f'Unsupported model "{args.model}"')

    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

In [ ]:
# configure optimizer
if args.strategy.startswith('colossalai'):
    actor_optim = HybridAdam(actor.parameters(), lr=5e-6)
    critic_optim = HybridAdam(critic.parameters(), lr=5e-6)
else:
    actor_optim = Adam(actor.parameters(), lr=5e-6)
    critic_optim = Adam(critic.parameters(), lr=5e-6)

In [ ]:
# setting the models
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = strategy.prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

In [ ]:
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=args.max_length,
                             do_sample=True,
                             DEFAULT_PAD_TOKEN = "[PAD]",
                             DEFAULT_EOS_TOKEN = "</s>",
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=3)
    print(outputs)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    return output


list_input = ["""이제 곧 졸업논문을 써야하는데 강사님이 논문 쓰는 법에 대해

구체적이고 상세한 예시를 통해 설명해주셔서 많은 도움이 되었습니다.

특히 강사님의 경험을 통한 설명이 인상깊었습니다.

앞으로 열심히 듣고 저도 논문 잘 쓸 수 있었으면 좋겠습니다. 감사합니다."""]

prompt = "아래의 텍스트를 분석하여 핵심 구문을 추출해줘\n"


list_input = [PROMPT_DICT['prompt_input'].format_map({'prompt': prompt,'input' : tmp}) for tmp in list_input]

list_prompt = [
    '불고기용 고기 한우에요?',
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_no_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

# for input_text in list_input:
#     output = actor.generate(input_text,
#                              max_length=args.max_length,
#                              do_sample=True,
#                              top_k=50,
#                              top_p=0.95,
#                              num_return_sequences=1)
#     print(output)
for input_text in list_input:
    output = generation(input_text)

In [ ]:
# prepare data

prompt = '아래의 텍스트를 분석하여 핵심 구문을 추출해줘\n'
with open(args.data_path_3_PPO, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_input = [tmp['input'] for tmp in list_data_dict]
    list_input = [PROMPT_DICT['prompt_input'].format_map({'prompt': prompt,'input' : tmp}) + '</s>' for tmp in list_input]
    #list_input2 = [PROMPT_DICT['prompt_input'].format_map({'prompt': prompt,'input' : tmp}) for tmp in list_input]

def tokenize_fn(texts):
    #prompt = '아래의 텍스트를 분석하여 핵심 구문을 추출해줘\n'
    #texts = [PROMPT_DICT['prompt_input'].format_map({'prompt': prompt,'input' : tmp}) for tmp in texts]
    #texts = [prompt + text + "<|endoftext|>" for text in texts]
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

print(list_input)
print('\n\n\n')
print(tokenize_fn(list_input))
print("#"*70)

In [ ]:
# configure trainer
trainer = PPOTrainer(strategy,
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=args.max_epochs,
                     train_batch_size=1,
                     tokenizer=tokenize_fn,
                     max_length=512,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

## train!
prompt = "아래의 텍스트를 분석하여 핵심 구문을 추출해줘\n"
trainer.fit(list_input, # 입력 prompt
            num_episodes=args.num_episodes,
            max_timesteps=args.max_timesteps,
            update_timesteps=args.update_timesteps)

## save
# save model checkpoint after fitting on only rank0
strategy.save_model(actor, os.path.join(args.output_dir, 'actor.pt'), only_rank0=True)
# save optimizer checkpoint on all ranks
strategy.save_optimizer(actor_optim,
                        os.path.join(args.output_dir, 'actor_optim_checkpoint_%d.pt' % (torch.cuda.current_device())),
                        only_rank0=False)

In [ ]:
## inference
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    print(input_ids)
    outputs = actor.generate(input_ids,
                             max_length=args.max_length,
                             do_sample=True,
                             pad_token_id=tokenizer.pad_token_id,
                             eos_token_id=tokenizer.eos_token_id,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=3)
    print(outputs)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print('#' * 70)
    print(output)
    return output


list_input = ["""이제 곧 졸업논문을 써야하는데 강사님이 논문 쓰는 법에 대해

구체적이고 상세한 예시를 통해 설명해주셔서 많은 도움이 되었습니다.

특히 강사님의 경험을 통한 설명이 인상깊었습니다.

앞으로 열심히 듣고 저도 논문 잘 쓸 수 있었으면 좋겠습니다. 감사합니다."""
               ]

prompt = "아래의 텍스트를 분석하여 핵심 구문을 추출해줘\n"

list_input = [PROMPT_DICT['prompt_input'].format_map({'prompt': prompt,'input' : tmp}) for tmp in list_input]

# list_result2 = generator(list_input2, **generation_args)
# for prompt, result in zip(list_input, list_result2):
#     print(('#'*70))
#     if mode == 'text-generation':
#       print(('completion: %s'%(result[0]['generated_text'])))
#     else:
#       print(result['summary_text'])

# list_prompt = [PROMPT_DICT['prompt_no_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

for input_text in list_input:
    output = generation(input_text)

#### 좋은글 생성기와 좋은글 채점기로 강화학습을 하여 ChatGPT-replica를 학습했습니다.  
#### ``output_3_PPO`` 폴더에 학습된 모델이 저장되어 있습니다.  
#### 끝~!

## inference PPO actor
- [ref](https://github.com/hpcaitech/ColossalAI/blob/main/applications/ChatGPT/examples/inference.py)

In [ ]:
# import
import argparse

import torch
torch.cuda.empty_cache()
from chatgpt.models.bloom import BLOOMActor
from chatgpt.models.gpt import GPTActor
from chatgpt.models.opt import OPTActor
from transformers import AutoTokenizer
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer

# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input":
    ("Below is an instruction that describes a task, paired with an input that provides further context.\n"
     "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n"
     "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n"
     "### Instruction(명령어):\n{prompt}\n\n### Input(입력):\n{input}\n\n### Response(응답):"
     ),
    "prompt_no_input":
    ("Below is an instruction that describes a task.\n"
     "아래는 작업을 설명하는 명령어입니다.\n\n"
     "Write a response that appropriately completes the request.\n명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.\n\n"
     "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"),
}

In [ ]:
# define argment
parser = argparse.ArgumentParser()
parser.add_argument('--model',
                    default='gpt2',
                    choices=['gpt2', 'bloom', 'opt'])
# We suggest to use the pretrained model from HuggingFace, use pretrain to configure model
parser.add_argument('--pretrain', type=str, default=None)
parser.add_argument('--model_path', type=str, default=None)
parser.add_argument('--input',
                    type=str,
                    default='Question: How are you ? Answer:')
parser.add_argument('--max_length', type=int, default=128)
args_inference = parser.parse_args([])

args_inference.model = 'gpt2'
args_inference.pretrain = 'skt/kogpt2-base-v2'
args_inference.model_directory = './output_3_PPO'
args_inference.model_path = os.path.join(args_inference.model_directory, 'actor.pt')

# configure model, tokenizer
if args_inference.model == 'gpt2':
    actor = GPTActor(pretrained=args_inference.pretrain).to(torch.cuda.current_device())
    # tokenizer = GPT2Tokenizer.from_pretrained(args_inference.pretrain)
    # tokenizer.pad_token = tokenizer.eos_token
    tokenizer = AutoTokenizer.from_pretrained(args_inference.pretrain,
                                              padding_side="right",
                                              model_max_length=512)
    tokenizer.add_special_tokens({
        "eos_token": DEFAULT_EOS_TOKEN,
        "bos_token": DEFAULT_BOS_TOKEN,
        "unk_token": DEFAULT_UNK_TOKEN,
    })
    tokenizer.pad_token = tokenizer.eos_token

elif args_inference.model == 'bloom':
    actor = BLOOMActor(pretrained=args_inference.pretrain).to(
        torch.cuda.current_device())
    tokenizer = AutoTokenizer.from_pretrained('bigscience/bloom-560m')
    tokenizer.pad_token = tokenizer.eos_token
elif args_inference.model == 'opt':
    actor = OPTActor(pretrained=args_inference.pretrain).to(torch.cuda.current_device())
    tokenizer = AutoTokenizer.from_pretrained('facebook/opt-350m')
else:
    raise ValueError(f'Unsupported model "{args_inference.model}"')

state_dict = torch.load(args_inference.model_path, map_location='cpu');
actor.model.load_state_dict(state_dict);

actor.eval();

In [ ]:
## inference
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=args_inference.max_length,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print('#' * 70)
    print(output)
    return output


list_prompt = [
    '불고기용 고기 한우에요?',
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_no_input'].format_map({'prompt': tmp}) for tmp in list_prompt]


# for input_text in list_prompt:
#     output = generation(input_text)

#### 좋은글 생성기와 좋은글 채점기로 강화학습을 하여 ChatGPT-replica를 학습했습니다.  
#### ``output_3_PPO`` 폴더에 학습된 모델이 저장되어 있습니다.  
#### 끝~!

In [ ]:
팀장님 으쌰으쌰하세요

In [ ]:
헉 언제 쓰셨지 감삼다